# Usage in Python

- Python has support for working with different databases:
1. sqlite
2. mysql
3. postgres
4. etc.

## Psycopg
- Psycopg is the most popular PostgreSQL database adapter for python
- Psycopg 2 is mostly implemented in C
- Many Python types are supported and adapted to matching PostgreSQL data types

## Basic module usage

> python3 -m pip install psycopg2 

> create database python_db;

In [1]:
import psycopg2

#Connect to an existing database
conn = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)

#Open a cursor to perform database operations
cursor = conn.cursor()

#Execute a command: this creates a new table
cursor.execute("CREATE TABLE if not exists test (id serial PRIMARY KEY, num integer, data varchar);")

#Pass data to fill a query placeholders 
cursor.execute("INSERT INTO test (num, data) VALUES (%s, %s);", (100, "abc'def"))

# Query the database and obtain data as Python objects
cursor.execute("SELECT * FROM test;")
data = cursor.fetchone()

# Make the changes to the database persistent
conn.commit()

# Close communication with the database
#cursor.close()
#conn.close()
data

(1, 100, "abc'def")

In [2]:
cursor.execute("select * from test;")
cursor.fetchall()

[(1, 100, "abc'def"),
 (2, 100, "abc'def"),
 (4, 100, "abc'def"),
 (5, 100, "abc'def"),
 (6, 100, "abc'def"),
 (7, 100, "abc'def"),
 (8, 100, "abc'def"),
 (9, 100, "abc'def"),
 (10, 100, "abc'def"),
 (11, 22, 'BLA'),
 (12, 100, 'DATA'),
 (16, 100, "abc'def"),
 (17, 100, 'DATA'),
 (19, None, 'Bla'),
 (20, None, 'Bla'),
 (21, None, 'Bla'),
 (22, None, 'Bla'),
 (25, None, 'Bla'),
 (31, None, 'HELLO WORLD'),
 (32, None, "HELLO WORLD'); select * from test where '1' in ('1"),
 (33, 1, None),
 (34, 1000, None),
 (35, 1000, None),
 (36, 2000, None),
 (37, 3000, None),
 (38, 1000, None),
 (39, 2000, None),
 (40, 3000, None),
 (41, 100, "abc'def")]

- connect() creates a new database session
(a new connection instance is returned)

In [3]:
print(type(conn))

<class 'psycopg2.extensions.connection'>


- each connection/session can create new cursor instances
- cursor instances can execute database commands
- commands are sent by the following methods:
1. execute()
2. executemany()
- to retrieve data use following cursor methods:
1. fetchone() - gets one row of the query
2. fetchall() - gets all available rows
3. fetchmany() - gets as many rows as specified e.g. fetchmany(5)

### Access Data From a cursor
- You can have many cursors sharing the same connection to a database.


- a query can potentially match very large sets of data
- Read operations do not immediately return all values matching the query
- these operations rely on cursors
- cursors fetches data in batches to reduce memory consumption

- Closing the cursor frees resources associated to the queries.
- It would not eliminate the connection to the database itself.
(Therefore, there is no need for reauthentication)

- Cursors created from the same connection are not isolated
- i.e. any changes done to the database by a cursor are visible by other cursor


### Connections and Cursors
- Connections and cursors can be used as context managers and commits if no exception occurs. 
- the connection is not closed by the context
- the cursor is closed by the context

In [4]:
with conn:
    with conn.cursor() as curs:
        cursor.execute("CREATE TABLE if not exists test3 (id serial PRIMARY KEY, num integer, data varchar);")
        curs.execute("SELECT * FROM test;")
        curs.execute("SELECT count(*) FROM test;")
        print(curs.fetchall())
# curs.fetchall()

[(29,)]


### Passing parameters to SQL queries 
### execute and executemany()
- Execute a database operation 
- Parameters may be provided 
- For positional variables binding, the second argument must always be a sequence

In [5]:
with conn:
    with conn.cursor() as curs:
#INSERT INTO some_table (an_int, a_date, a_string)
#VALUES (21, 22, 'BLA');
        curs.execute("""
        insert into test (id, num, data)
        values (%s, %s, %s)
        """, (23, 22, 'BLA'))
        curs.execute("select * from test;")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA')]


- Named arguments are supported too using %(name)s placeholders in the query
- allows to specify the values in any order and to repeat the same value

In [6]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%(value1)s, %(value2)s)
        """, {'value1':100, 'value2': 'DATA'})
        curs.execute("select * from test;")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA'), (42, 100, 'DATA')]


### Context and context object
- shows that its the same connection

In [7]:
with conn as conn_val:
    with conn.cursor() as curs:
        curs.execute("select * from test;")
        print(curs.fetchall())
        print(conn_val)

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA'), (42, 100, 'DATA')]
<connection object at 0x7f25d7b80680; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>


In [8]:
with conn as conn_val:
    with conn.cursor() as curs:
        curs.execute("select * from test;")
        print(curs.fetchall())
        print(conn_val)
print(conn_val)
# conn.close()
# print(conn_val)

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA'), (42, 100, 'DATA')]
<connection object at 0x7f25d7b80680; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>
<connection object at 0x7f25d7b80680; dsn: 'user=postgres password=xxx dbname=python_db host=localhost', closed: 0>


### The problem with the query parameters - SQL Injection

In [9]:
with conn as conn_val:
    with conn.cursor() as curs:
        SQL = "INSERT INTO test (num) VALUES (%s);" # Note: no quotes, even without quotes
        user_input = ("1); select * from test where 1 in (1", ) #Malicious input
        curs.execute(SQL % user_input)
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA'), (42, 100, 'DATA'), (43, 1, None)]


In [10]:
with conn as conn_val:
    with conn.cursor() as curs:
        SQL = "INSERT INTO test (num) VALUES (%s);" # Note: no quotes, even without quotes
        user_input = ("1); select * from test where 1 in (1", ) #Malicious input
        curs.execute(SQL, user_input)# handling the quote; prevents injection
        print(curs.fetchall())

InvalidTextRepresentation: invalid input syntax for type integer: "1); select * from test where 1 in (1"
LINE 1: INSERT INTO test (num) VALUES ('1); select * from test where...
                                       ^


<span style=color:red;>Warning: Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. </span>

![](exploits_of_a_mom.png)

### executemany

In [11]:
with conn:
    with conn.cursor() as curs:
        nums = ((1,), (5,), (10,)) #tuple of tuple
        curs.executemany("INSERT INTO test (num) VALUES (%s)", nums)
        curs.execute("select * from test")
        print(curs.fetchall())


[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla'), (31, None, 'HELLO WORLD'), (32, None, "HELLO WORLD'); select * from test where '1' in ('1"), (33, 1, None), (34, 1000, None), (35, 1000, None), (36, 2000, None), (37, 3000, None), (38, 1000, None), (39, 2000, None), (40, 3000, None), (41, 100, "abc'def"), (23, 22, 'BLA'), (42, 100, 'DATA'), (43, 1, None), (44, 1, None), (45, 5, None), (46, 10, None)]


- Parameters are bounded to the query using the same rules described in the execute() method.
- In its current implementation this method is not faster than executing execute() in a loop

### Adaptation of Python values to SQL types
- There is default mapping to convert Python types into PostgreSQL types, and vice versa

- None <-->NULL
- bool <-->bool
- decimal <--> numeric
- str <--> varchar
- float <--> real or double
- int <--> smallint or integer or bigint
- date <--> date
- time <--> timetz

#### mogrify
- Return a query string after arguments binding. 
- The string returned is exactly the one that would be sent by execute() method.


In [12]:
with conn:
    with conn.cursor() as curs:
        query1 = curs.mogrify("INSERT INTO test (num, data) VALUES (10, 'bal');")
        query2 = curs.mogrify("""
        INSERT INTO test (num, data) VALUES (%s, %s);
        """, (10,'BLA'))
        print(query2)

b"\n        INSERT INTO test (num, data) VALUES (10, 'BLA');\n        "


- the returned value is always a byte string

### Constants adaptation
- Python None, True and False are converted into the proper SQL literals:

In [13]:

with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(None, True, False))
        print(query)

b'\n        SELECT NULL, true, false;'


### Numeric adaptation

In [14]:
from decimal import Decimal
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(10, 10.00, Decimal("10.00")))
        print(query)

b'\n        SELECT 10, 10.0, 10.00;'


- sometimes you may prefer to receive numeric data as float instead:
- you can configure an adapter to cast PostgreSQL numeric to Python float.


### Date/Time objects adaptation

In [15]:
from datetime import datetime
dt = datetime.now()
dt
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s, %s, %s;""",(dt,dt.date(),dt.time()))
        print(query)


b"\n        SELECT '2023-03-13T21:04:01.501091'::timestamp, '2023-03-13'::date, '21:04:01.501091'::time;"


### Lists adaptation


In [16]:
with conn:
    with conn.cursor() as curs:
        query = curs.mogrify("""
        SELECT %s;""",([10, 20, 30], ))
        print(query)

b'\n        SELECT ARRAY[10,20,30];'


### Transactions

- Transactions are handled by the connection class
- the first time a command is sent to the database, a new transaction is created.
- All db commands will be executed in the context of the same transaction
- all cursors created by the same connection share the same transaction

Should any command fail, the transaction will be aborted 
- no further command will be executed until rollback is called.

In [127]:
conn1 = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)

conn2 = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)


In [153]:
curs1 = conn1.cursor()
curs2 = conn1.cursor()
curs2_1 = conn2.cursor()
conn1.rollback()
try:
    curs1.execute('select * from bla')
except Exception as e:
    print(e)

curs2_1.execute('select * from test')
print(curs2_1.fetchall())

curs2.execute('select * from test')
curs2.fetchall()

relation "bla" does not exist
LINE 1: select * from bla
                      ^

[(1, 100, "abc'def"), (2, 22, 'BLA'), (3, 100, 'DATA'), (8, 123, 'HELLO'), (9, 123, 'HELLO'), (10, 123, 'HELLO'), (11, 123, 'HELLO2'), (15, 123, 'HELLO2'), (16, 123, 'HELLO2'), (17, 123, 'HELLO2')]


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


- Postgres relies heavily on transactions to keep data consistent across concurrent connections and parallel activities.
- transactions allow a database to implement the ACID properties:
the db must be able:
1. to handle units of work on its whole (Atomicity)
2. to store data without inter-mixed changes to the data (consistency)
3. to store data in a way that concurrency actions are executed as if they were
alone (isolation) 
2. to store data in a permanent way (Durability)

- An atomic transaction is an indivisible and irreducible series of database operations such that either all occurs, or nothing occurs.

- every action issued against the database is executed within a transaction
- any transaction is assigned a unique number: *xid*
- you can get the xid by inspecting the hidden column *xmin*

In [166]:
SQL_xid = 'select txid_current()'
SQL_insert = "insert into test(num, data) values(123, 'HELLO')"
SQL_xmin = 'select xmin, txid_current(),  * from test'

In [170]:
conn1.rollback()
curs1.execute(SQL_insert)
curs1.execute(SQL_insert)
curs2_1.execute(SQL_insert) # will not be committed
conn1.commit() #old transaction is committed and new transaction starts
curs1.execute(SQL_insert)
curs1.execute(SQL_insert)
curs2.execute(SQL_insert)
curs1.execute(SQL_xmin)
curs1.fetchall()

[('3005', 3043, 1, 100, "abc'def"),
 ('3007', 3043, 2, 22, 'BLA'),
 ('3008', 3043, 3, 100, 'DATA'),
 ('3016', 3043, 8, 123, 'HELLO'),
 ('3017', 3043, 9, 123, 'HELLO'),
 ('3018', 3043, 10, 123, 'HELLO'),
 ('3019', 3043, 11, 123, 'HELLO2'),
 ('3023', 3043, 15, 123, 'HELLO2'),
 ('3024', 3043, 16, 123, 'HELLO2'),
 ('3025', 3043, 17, 123, 'HELLO2'),
 ('3034', 3043, 30, 123, 'HELLO WORLD'),
 ('3034', 3043, 31, 123, 'HELLO WORLD'),
 ('3035', 3043, 32, 123, 'HELLO WORLD'),
 ('3037', 3043, 33, 123, 'HELLO'),
 ('3037', 3043, 34, 123, 'HELLO'),
 ('3037', 3043, 35, 123, 'HELLO'),
 ('3039', 3043, 39, 123, 'HELLO'),
 ('3039', 3043, 40, 123, 'HELLO'),
 ('3039', 3043, 41, 123, 'HELLO'),
 ('3041', 3043, 45, 123, 'HELLO'),
 ('3041', 3043, 46, 123, 'HELLO'),
 ('3043', 3043, 48, 123, 'HELLO'),
 ('3043', 3043, 49, 123, 'HELLO'),
 ('3043', 3043, 50, 123, 'HELLO')]

- to do multiple transactions in psql:
```
begin;
insert into test(num, data) values(123, 'HELLO WORLD')
insert into test(num, data) values(123, 'HELLO WORLD')
commit;
``` 
- rollback: postgres throws away our changes and keeps the pre.transaction state

- when postgres issues an error, it aborts the current transaction
- it will not honor any following command nor commit and will issue a rollback:

```
begin;
select * from table_that_does_not_exists; --causes error 
select * from test;
commit;
``` 

### Connection Pooling

- connections pools are cached database connections
- those connections are created and maintained to get reused
- main benefit: performance improvement

Connection pooling is the process of having a pool of active connections on the backend servers. 
- These can be used any time a user sends a request.
- The server will assign an active connection to the user.

![](pooling.png)

### Example

### SimpleConnectionPool:



```
conn = psycopg2.connect(
    host="localhost",
    database="python_db",
    user="postgres",
    password="password"
)
```

In [178]:
from psycopg2.pool import SimpleConnectionPool

pool = SimpleConnectionPool(minconn=1, maxconn=20,
                        host="localhost",
                        database="python_db",
                        user="postgres",
                        password="password"
                        )

In [91]:
SQL_active_db = "select usename, client_addr, state from pg_stat_activity where datname='python_db'"


In [55]:
SQL_killall = """
select pg_terminate_backend(pid) from pg_stat_activity where datname='python_db'
"""

In [63]:
SQL_active = "select datname from pg_stat_activity"

In [174]:
SQL_select = "Select * from test" 
SQL_insert = "insert into test(num, data) values(123, 'HELLO POOL')"
SQL_xmin = 'select xmin, txid_current(),  * from test'

In [182]:
conn = pool.getconn()
conn2 = pool.getconn()
curs = conn.cursor()
curs2 = conn.cursor()
curs2_1 = conn2.cursor()
curs.execute(SQL_insert)
curs2.execute(SQL_insert)
curs2_1.execute(SQL_insert)
conn.commit() #To save any changes in the database
curs.execute(SQL_xmin)
data = curs.fetchall()
# print(curs.closed)
# curs.close()
# print(curs.closed)

# release the connection object and send back to connection pool
pool.putconn(conn)
data

[('3005', 3060, 1, 100, "abc'def"),
 ('3007', 3060, 2, 22, 'BLA'),
 ('3008', 3060, 3, 100, 'DATA'),
 ('3016', 3060, 8, 123, 'HELLO'),
 ('3017', 3060, 9, 123, 'HELLO'),
 ('3018', 3060, 10, 123, 'HELLO'),
 ('3019', 3060, 11, 123, 'HELLO2'),
 ('3023', 3060, 15, 123, 'HELLO2'),
 ('3024', 3060, 16, 123, 'HELLO2'),
 ('3025', 3060, 17, 123, 'HELLO2'),
 ('3034', 3060, 30, 123, 'HELLO WORLD'),
 ('3034', 3060, 31, 123, 'HELLO WORLD'),
 ('3035', 3060, 32, 123, 'HELLO WORLD'),
 ('3037', 3060, 33, 123, 'HELLO'),
 ('3037', 3060, 34, 123, 'HELLO'),
 ('3037', 3060, 35, 123, 'HELLO'),
 ('3039', 3060, 39, 123, 'HELLO'),
 ('3039', 3060, 40, 123, 'HELLO'),
 ('3039', 3060, 41, 123, 'HELLO'),
 ('3041', 3060, 45, 123, 'HELLO'),
 ('3041', 3060, 46, 123, 'HELLO'),
 ('3044', 3060, 51, 123, 'HELLO'),
 ('3046', 3060, 52, 123, 'HELLO POOL'),
 ('3046', 3060, 53, 123, 'HELLO POOL'),
 ('3048', 3060, 54, 123, 'HELLO POOL'),
 ('3048', 3060, 55, 123, 'HELLO POOL'),
 ('3050', 3060, 56, 123, 'HELLO POOL'),
 ('3050', 3060,

### Simple Pooling with Context Manager

In [119]:
SQL_insert = "insert into test(num, data) values(123, 'HELLO2')"

In [120]:
with pool.getconn() as conn:
    with conn.cursor() as curs:
        # curs.execute(SQL_active_db) 
        curs.execute(SQL_insert) 
        curs.execute(SQL_select)
        print(curs.fetchall())
pool.putconn(conn)


[(1, 100, "abc'def"), (2, 22, 'BLA'), (3, 100, 'DATA'), (8, 123, 'HELLO'), (9, 123, 'HELLO'), (10, 123, 'HELLO'), (11, 123, 'HELLO2')]
